### Variable cheat sheet:
* *file_raw:* DataFrame obtained from the original CSV files in /data/raw.
* *file_preinterim:* DataFrame that has passed through our first two cleaning steps.
* *file_interim:* DataFrame that has passed through all three cleaning steps, and which has been saved in /data/interim.
* *file_MultiIndex:* MultiIndex object.
* *file:* MultIindex DataFrame which has been saved in /data/processed.

# Raw data:
### Three datasets in CSV format, corresponding to the pandemic's three semesters: January-July 2020, August-December 2020, and January-July 2021.
### Each row contains pandemic-relevant demographic information about a municipality in Brazil. Almost every one of the 5570 municipalities are covered.

# Objectives:
### To practice the application of descriptive and inferential Statistics 101 tools to COVID-19 data.
### In specific, we will look at mortality rate statistics across time and across regions in Brazil.

In [1]:
import os
import time
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
# Loading from /data/raw.
load_path = 'data' + os.sep + 'raw' + os.sep
title = 'HIST_PAINEL_COVIDBR'
download_date = '08jun2021'

start = time.time()
JanJul2020_raw = pd.read_csv(f'{load_path}{title}_2020_Parte1_{download_date}.csv', sep=';', low_memory=False)
AugDec2020_raw = pd.read_csv(f'{load_path}{title}_2020_Parte2_{download_date}.csv', sep=',', low_memory=False)
JanJul2021_raw = pd.read_csv(f'{load_path}{title}_Parte3_{download_date}.csv', sep=',', low_memory=False)
end = time.time()

print(f'Original csv import time: {end - start :.2f} secs')

Original csv import time: 8.23 secs


### Let us look at a few arbitrary rows.

In [3]:
JanJul2020_raw.loc[4929:4934, : ]

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
4929,Norte,RO,Cabixi,11,110003.0,11006.0,CONE SUL,2020-07-29,31,5312.0,44,3,3,0,NaN,NaN,0.0
4930,Norte,RO,Cabixi,11,110003.0,11006.0,CONE SUL,2020-07-30,31,5312.0,44,0,3,0,NaN,NaN,0.0
4931,Norte,RO,Cabixi,11,110003.0,11006.0,CONE SUL,2020-07-31,31,5312.0,44,0,3,0,NaN,NaN,0.0
4932,Norte,RO,Cacoal,11,110004.0,11002.0,CAFE,2020-03-27,13,85359.0,0,0,0,0,NaN,NaN,0.0
4933,Norte,RO,Cacoal,11,110004.0,11002.0,CAFE,2020-03-28,13,85359.0,0,0,0,0,NaN,NaN,0.0
4934,Norte,RO,Cacoal,11,110004.0,11002.0,CAFE,2020-03-29,14,85359.0,0,0,0,0,NaN,NaN,0.0


### There are  4000+ rows containg aggregate data corresponding to states, regions, and the country as a whole. Our first step is to drop those.
### Our second step is to remove all information not pertaining to mortality. The cell below carries out our first two steps.

In [6]:
def drop_non_municipalities(df):
    return df[df['municipio'].notnull()]

def drop_columns(df):
    new_df = df.loc[ : , ['municipio', 'data', 'populacaoTCU2019', 'obitosAcumulado', 'casosAcumulado']]
    new_df.columns = ['City', 'Date', 'Population', 'Total deaths', 'Total cases']
    return new_df

JanJul2020_preinterim = drop_columns(drop_non_municipalities(JanJul2020_raw))
AugDec2020_preinterim = drop_columns(drop_non_municipalities(AugDec2020_raw))
JanJul2021_preinterim = drop_columns(drop_non_municipalities(JanJul2021_raw))

JanJul2020_preinterim.loc[4929:4934, : ]

,City,Date,Population,Total deaths,Total cases
4929,Cabixi,2020-07-29,5312.0,3,44
4930,Cabixi,2020-07-30,5312.0,3,44
4931,Cabixi,2020-07-31,5312.0,3,44
4932,Cacoal,2020-03-27,85359.0,0,0
4933,Cacoal,2020-03-28,85359.0,0,0
4934,Cacoal,2020-03-29,85359.0,0,0


### Our third step is to transform total deaths and population into deaths per million ("death rate").
### This process is very time-consuming and has been commented-out below. The result has been pickled in /data/interim.

In [7]:
def get_death_rate(df):
    death_rate_series = pd.Series([0] * len(df.index))
    for index, row in df.iterrows():
        death_rate = (int(row['Total deaths']) / int(row['Population'])) * 1000000 
        death_rate_series[index] = death_rate
    df.loc[ : , 'Death rate'] = death_rate_series
    return df


def process_and_save(df, filename):
    start = time.time()
    df = get_death_rate(df)
    end = time.time()
    print(f'Death rate obtainment time: {end - start} secs')
    save_path = 'data' + os.sep + 'interim' + os.sep
    df.to_pickle(f'{save_path}{filename}')

In [9]:
process_and_save(JanJul2020_preinterim, 'JanJul2020')
# 441 secs

Death rate obtainment time: 493.10004234313965 secs


In [10]:
process_and_save(AugDec2020_preinterim, 'AugDec2020')
# 1087 secs

Death rate obtainment time: 741.1877574920654 secs


In [11]:
process_and_save(JanJul2021_preinterim, 'JanJul2021')
# 868 secs

Death rate obtainment time: 945.5933654308319 secs


In [12]:
save_path = 'data' + os.sep + 'interim' + os.sep
JanJul2020_interim = pd.read_pickle(f'{save_path}JanJul2020')
AugDec2020_interim = pd.read_pickle(f'{save_path}AugDec2020')
JanJul2021_interim = pd.read_pickle(f'{save_path}JanJul2021')
JanJul2020_interim

,City,Date,Population,Total deaths,Total cases,Death rate
4551,Alta Floresta D'Oeste,2020-03-27,22945.0,0,0,0.000000
4552,Alta Floresta D'Oeste,2020-03-28,22945.0,0,0,0.000000
4553,Alta Floresta D'Oeste,2020-03-29,22945.0,0,0,0.000000
4554,Alta Floresta D'Oeste,2020-03-30,22945.0,0,0,0.000000
4555,Alta Floresta D'Oeste,2020-03-31,22945.0,0,0,0.000000
...,...,...,...,...,...,...
714476,Brasília,2020-07-27,3015268.0,1339,98480,444.073296
714477,Brasília,2020-07-28,3015268.0,1391,100726,461.318861
714478,Brasília,2020-07-29,3015268.0,1419,102342,470.604935
714479,Brasília,2020-07-30,3015268.0,1444,104442,478.896072


### We wish to query the DataFrame informing a city and a date, obtaining the accumulated death rate in return. A MultiIndex DataFrame is used for this purpose.

In [13]:
def get_MultiIndex(df):
    cities = df['City'].unique()
    dates = df['Date'].unique()
    tuples = list(zip(df['City'], df['Date']))
    return pd.MultiIndex.from_tuples(tuples, names=['City', 'Date'])

start = time.time()
JanJul2020_MultiIndex = get_MultiIndex(JanJul2020_interim)
AugDec2020_MultiIndex = get_MultiIndex(AugDec2020_interim)
JanJul2021_MultiIndex = get_MultiIndex(JanJul2021_interim)
end = time.time()
print(f'MultiIndex obtainment time: {end - start} secs')

MultiIndex obtainment time: 3.6557252407073975 secs


In [24]:
JanJul2020_interim.loc[ : , ['Population', 'Total deaths', 'Total cases', 'Death rate']].to_numpy()

array([[2.29450000e+04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.29450000e+04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.29450000e+04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.01526800e+06, 1.41900000e+03, 1.02342000e+05, 4.70604935e+02],
       [3.01526800e+06, 1.44400000e+03, 1.04442000e+05, 4.78896072e+02],
       [3.01526800e+06, 1.46900000e+03, 1.06292000e+05, 4.87187209e+02]])

In [25]:
def df_constructor(df, multi_index):
    return pd.DataFrame(data=df.loc[ : , ['Population', 'Total deaths', 'Total cases', 'Death rate']].to_numpy(), index=multi_index, columns=['Population', 'Total deaths', 'Total cases', 'Death rate'])

start = time.time()
JanJul2020 = df_constructor(JanJul2020_interim, JanJul2020_MultiIndex)
AugDec2020 = df_constructor(AugDec2020_interim, AugDec2020_MultiIndex)
JanJul2021 = df_constructor(JanJul2021_interim, JanJul2021_MultiIndex)
end = time.time()
print(f'MultiIndex DataFrame obtainment time: {end - start} secs')

MultiIndex DataFrame obtainment time: 0.1375277042388916 secs


In [26]:
JanJul2020

Population  Total deaths  Total cases  \
City                  Date                                                
Alta Floresta D'Oeste 2020-03-27     22945.0           0.0          0.0   
                      2020-03-28     22945.0           0.0          0.0   
                      2020-03-29     22945.0           0.0          0.0   
                      2020-03-30     22945.0           0.0          0.0   
                      2020-03-31     22945.0           0.0          0.0   
...                                      ...           ...          ...   
Brasília              2020-07-27   3015268.0        1339.0      98480.0   
                      2020-07-28   3015268.0        1391.0     100726.0   
                      2020-07-29   3015268.0        1419.0     102342.0   
                      2020-07-30   3015268.0        1444.0     104442.0   
                      2020-07-31   3015268.0        1469.0     106292.0   

                                  Death rate  
City                  Date                    
Alta Floresta D'Oeste 2020-03-27    0.000000  
                      2020-03-28    0.000000  
                      2020-03-29    0.000000  
                      2020-03-30    0.000000  
                      2020-03-31    0.000000  
...                                      ...  
Brasília              2020-07-27  444.073296  
                      2020-07-28  461.318861  
                      2020-07-29  470.604935  
                      2020-07-30  478.896072  
                      2020-07-31  487.187209  

[707390 rows x 4 columns]

### Now that we have our final database, we can easily obtain death rate information.

In [27]:
def get_value(df, place, date):
    return df.loc[place].loc[date]['Death rate']

get_value(JanJul2020, 'Brasília', '2020-07-30')

478.8960715929729

### We finish by saving our results.

In [28]:
save_path = 'data' + os.sep + 'processed' + os.sep
JanJul2020.to_pickle(f'{save_path}JanJul2020')
AugDec2020.to_pickle(f'{save_path}AugDec2020')
JanJul2021.to_pickle(f'{save_path}JanJul2021')

# Problem in the original database.
### There is, however, something funny in the original database. In the raw, original CSV databases for AugDec2020 and JanJul2021, some rows were dislodged one column to the left. This led us to obtain nonsensical dates and absurd death rates (over 10 million deaths per million inhabitants). Here is an example.

In [29]:
AugDec2020.loc['Manaus'].head(5)

,Population,Total deaths,Total cases,Death rate
Date,,,,
ENTORNO E ALTO RIO NEGRO,31.0,395.0,2182763.0,12741935.0
ENTORNO E ALTO RIO NEGRO,32.0,43.0,2182763.0,1343750.0
ENTORNO E ALTO RIO NEGRO,32.0,22.0,2182763.0,687500.0
ENTORNO E ALTO RIO NEGRO,32.0,347.0,2182763.0,10843750.0
ENTORNO E ALTO RIO NEGRO,32.0,272.0,2182763.0,8500000.0


### To fix this, let us get a list of all municipalities containing bad date formats.

In [30]:
def get_bad_cities(df):
    dates  = df.index.levels[1]
    bad_dates = list()
    for date in dates:
        if not date[0].isdigit():
            bad_dates.append(date)
    cities = df.index.levels[0]
    bad_cities = list()
    for bad_date in bad_dates:
        for city in cities:
            try:
                df.loc[city].loc[bad_date]
                bad_cities.append(city)
            except:
                pass
    return bad_cities

In [31]:
JanJul2020_badcitylist = get_bad_cities(JanJul2020)
AugDec2020_badcitylist = get_bad_cities(AugDec2020)
JanJul2021_badcitylist = get_bad_cities(JanJul2021)
print(f"Bad cities for Jan-Jul 2020: {JanJul2020_badcitylist}")
print(f"Bad cities for Aug-Dec 2020: {AugDec2020_badcitylist}")
print(f"Bad cities for Jan-Jul 2021: {JanJul2021_badcitylist}")

Bad cities for Jan-Jul 2020: []
Bad cities for Aug-Dec 2020: ['Autazes', 'Barcelos', 'Careiro', 'Careiro da Várzea', 'Iranduba', 'Manaquiri', 'Manaus', 'Nova Olinda do Norte', 'Presidente Figueiredo', 'Rio Preto da Eva', 'Santa Isabel do Rio Negro', 'São Gabriel da Cachoeira']
Bad cities for Jan-Jul 2021: ['Autazes', 'Barcelos', 'Careiro', 'Careiro da Várzea', 'Iranduba', 'Manaquiri', 'Manaus', 'Nova Olinda do Norte', 'Presidente Figueiredo', 'Rio Preto da Eva', 'Santa Isabel do Rio Negro', 'São Gabriel da Cachoeira']


### We can now extract the original data for exactly those cities.

In [32]:
def get_correct_df(df, badcitylist):
    df_good = pd.DataFrame()
    for bad_city in badcitylist:
        df_subset = df[df['municipio'] == bad_city]
        df_good = df_good.append(df_subset)
    return df_good

In [33]:
AugDec2020_badcities = get_correct_df(AugDec2020_raw, AugDec2020_badcitylist)
JanJul2021_badcities = get_correct_df(JanJul2021_raw, JanJul2021_badcitylist)
AugDec2020_badcities.head(1)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana,Unnamed: 17
16678,Norte,AM,Autazes,13,130030.0,13001.0,MANAUS,ENTORNO E ALTO RIO NEGRO,2020-08-01,31.0,39565,1160,3,35,0.0,NaN,NaN,1.0


### We again drop columns containing data irrelevant for us, but now taking into account that the data is shifted to the right.

In [34]:
def drop_columns_2(df):
    new_df = df.loc[ : , ['municipio', 'semanaEpi', 'casosAcumulado', 'casosNovos', 'obitosNovos']]
    new_df.columns = ['City', 'Date', 'Population', 'Total cases', 'Total deaths']
    return new_df

AugDec2020_preinterim_badcities = drop_columns_2(AugDec2020_badcities)
JanJul2021_preinterim_badcities = drop_columns_2(JanJul2021_badcities)
AugDec2020_preinterim_badcities.head(1)

,City,Date,Population,Total cases,Total deaths
16678,Autazes,2020-08-01,39565,1160,35


### We now obtain the correct dates and death rates and create a MultiIndex DataFrame.

In [35]:
AugDec2020_interim_badcities = get_death_rate(AugDec2020_preinterim_badcities)
JanJul2021_interim_badcities = get_death_rate(JanJul2021_preinterim_badcities)
AugDec2020_interim_badcities.head(1)

,City,Date,Population,Total cases,Total deaths,Death rate
16678,Autazes,2020-08-01,39565,1160,35,884.620245


In [36]:
AugDec2020_MultiIndex_badcities = get_MultiIndex(AugDec2020_interim_badcities)
JanJul2021_MultiIndex_badcities = get_MultiIndex(JanJul2021_interim_badcities)

In [37]:
AugDec2020_badcities_final = df_constructor(AugDec2020_interim_badcities, AugDec2020_MultiIndex_badcities)
JanJul2021_badcities_final = df_constructor(JanJul2021_interim_badcities, JanJul2021_MultiIndex_badcities)
AugDec2020_badcities_final.head(5)

Population  Total deaths  Total cases  Death rate
City    Date                                                         
Autazes 2020-08-01     39565.0          35.0       1160.0  884.620245
        2020-08-02     39565.0          35.0       1160.0  884.620245
        2020-08-03     39565.0          35.0       1160.0  884.620245
        2020-08-04     39565.0          35.0       1163.0  884.620245
        2020-08-05     39565.0          36.0       1175.0  909.895109

### We end by appending the new, correct DataFrame to our previous one, and dropping the rows containing incorrect data.

In [38]:
AugDec2020 = AugDec2020.append(AugDec2020_badcities_final)
JanJul2021 = JanJul2021.append(JanJul2021_badcities_final)

AugDec2020 = AugDec2020.drop(' ENTORNO E ALTO RIO NEGRO', level='Date')
JanJul2021 = JanJul2021.drop(' ENTORNO E ALTO RIO NEGRO', level='Date')

AugDec2020.loc['Manaus'].head(5)

,Population,Total deaths,Total cases,Death rate
Date,,,,
2020-08-01,2182763.0,2016.0,35987.0,923.600043
2020-08-02,2182763.0,2019.0,36030.0,924.974448
2020-08-03,2182763.0,2021.0,36052.0,925.890717
2020-08-04,2182763.0,2027.0,36399.0,928.639527
2020-08-05,2182763.0,2036.0,36671.0,932.762742


### We pickle our new datasets. The end.

In [39]:
save_path = 'data' + os.sep + 'processed' + os.sep
AugDec2020.to_pickle(f'{save_path}AugDec2020')
JanJul2021.to_pickle(f'{save_path}JanJul2021')